In [1]:
__author__ = "Charlie Mydlarz"
__version__ = "0.1"
__status__ = "Development"
__adaptation__ = "Alejandro Porcel"

import pyaudio
import time
import datetime
import numpy as np
import wave
import os, sys
import pandas as pd

In [2]:
# Declare variable global so other threads can make use of it
global wavefile, recording, collecting, sample_counter
global collected_data, wavename, RMS_thereshold, silence_counter
global max_silence, rms_list, tstamp_list, recording_list, file_counter
global file_frames_size, rms_collector, buffers_per_second


sample_rate = 44100                  # Sample rate of audio device
frames_per_buffer = 2100             # Number of audio frames delivered per hardware buffer return
buffers_per_second = sample_rate / frames_per_buffer 
channels = 1                         # Number of audio channels (1 = mono)
#fname = str(time.time()) + '.wav'   # Output wave filename using current UTC time
total_duration = 43200               # Total length of wave file (12h=43200s)
device_id = -1                       # Default audio input device ID
recording = True                     # Boolean check to hold while wait loop
collecting = False                   # Boolean check to be collecting
sample_counter = 0                   # Counts the Frames Gotten
collected_data = []                  # Collected Data
RMS_thereshold = pow(10,-36/20)      # Thereshold level for RMS to be recorded (-36 dB)
silence_counter = 0                  # Counts Silences
max_silence = buffers_per_second - 1 # Gives two silence buffers (or below RMS thereshold) to the sample as max
rms_list = []                        # List that have RMS to save
tstamp_list = []                     # List that have timestamp
recording_list = []                  # List that have recording flag
file_counter = 0                     # Counter of audio buffers to check if they should be recorded
file_frames_size = 21                # Number of audio buffers that the continuously record to csv file 
rms_collector = []                   # Sublist to collect all rms in frame to average them

# Initialize PyAudio
pa = pyaudio.PyAudio()

In [3]:
#Output wave filename using current local Time format = YearMonthDayHourMinuteSecondMicrosecond
def tstamp():
    return datetime.datetime.now().strftime('%Y%m%d%H%M%S%f')
def fname():
    return tstamp() + '.wav'

In [4]:
def select_audio_device():
#    """ This method will list all the devices connected to host machine along with its index value"""
    print('Index\tValue\n===============')

    for i in range(pa.get_device_count()):
        devinfo = pa.get_device_info_by_index(i)

        # Convert dictionary key:value pair to a tuple
        for k in list(devinfo.items()):
            name, value = k

            if 'name' in name:
                print i, '\t', value

    try:
        return int(raw_input('\nEnter input device index: '))
    except ValueError:
        print "Not a valid device, falling back to default device"
        return -1

In [5]:
def recorder_callback(in_data, frame_count, time_info, status_flags):
#    """ This method is called whenever the audio device has acquired the number of audio samples
#    defined in 'frames_per_buffer'. This method is called by a different thread to the main thread so some variables
#    need to be declared global when altered within it. Avoid any heavy number crunching within this method as it
#    can disrupt audio I/O if it blocks for too long.
#
#    Args:
#        in_data (str): Byte array of audio data from the audio input device.
#        frame_count (int): Number of audio samples/frames received, will be equal to 'frames_per_nuffer'.
#        time_info (dict): Dictionary of time information for audio sample data
#        status_flags (long): Flag indicating any errors in audio capture
#    """
    global wavefile, recording, collecting, sample_counter
    global collected_data, wavename, RMS_thereshold, silence_counter
    global max_silence, rms_list, tstamp_list, recording_list, file_counter
    global file_frames_size, rms_collector, buffers_per_second

    # Convert byte array data into numpy array with a range -1.0 to +1.0
    audio_data = np.fromstring(in_data, dtype=np.int16) / 32767.0
    
    sample_counter = sample_counter + frames_per_buffer
    # Calculate root mean squared of audio buffer
    rms = np.sqrt(np.mean(np.square(audio_data)))
    
    #increase the file recorder with one more buffer
    file_counter = file_counter + 1
    rms_collector.append(rms)
    
    # Check Counter for write in log file
    if file_counter >= file_frames_size:
        file_counter = 0
        rms_list.append(np.sqrt(np.mean(np.square(rms_collector))))
        tstamp_list.append(tstamp())
        recording_list.append(rms >= RMS_thereshold)
        rms_collector = []
    # Check RMS thereshold
    if rms >= RMS_thereshold:
        silence_counter = 0

        if not collecting:
            wavename = fname()
            collecting = True
            wavefile = wave.open(wavename, 'wb')
    
            # Set number of input channels
            wavefile.setnchannels(channels)

            # Set sample width = 2, as each 16bit sample value consists of 2 bytes: http://wavefilegem.com/how_wave_files_work.html
            wavefile.setsampwidth(pa.get_sample_size(pyaudio.paInt16))

            # Set sample rate at 44,100 sample values per second
            wavefile.setframerate(sample_rate)
            collected_data = []
        collected_data.append(in_data)
    else:
        n = len(collected_data) 
        if collecting:
            silence_counter += 1
            if silence_counter <= max_silence:
                #collects one silence 
                collected_data.append(in_data)
            else:
                if n + 1 - silence_counter >= buffers_per_second: # decrease by silence counter because don't want to record with too many silences 
                    print "Collected at {0}, {1} samples, with {2} silences".format(wavename,n,silence_counter)
                    for sample_data in collected_data:
                        wavefile.writeframes(sample_data)
                    wavefile.close()
                else:
                    #print "Error Collecting at {0}, {1} samples".format(wavename,n)
                    wavefile.close()
                    os.remove(wavename)
                silence_counter = 0
                #reset data collection
                collected_data = []
                collecting = False
        else:
            silence_counter = 0
            #reset data collection
            collected_data = []
            collecting = False
    return None, pyaudio.paContinue

In [6]:
# List and select audio input device
device_id = select_audio_device()

Index	Value
0 	Built-in Microph
1 	Built-in Output

Enter input device index: 0


In [7]:
# Initialize recording stream object passing all predefined settings
recorder = pa.open(start=False,
                   input_device_index=device_id,
                   format=pyaudio.paInt16,
                   channels=channels,
                   rate=sample_rate,
                   input=True,
                   frames_per_buffer=frames_per_buffer,
                   stream_callback=recorder_callback)

In [8]:
# Start recording stream, triggering hardware buffer fill and callback
recorder.start_stream()

# Hold script in loop waiting for wave file to meet desired file length in seconds
while recording:
    # If wavefile length is equal to the duration given then change recording flag
    if sample_counter > total_duration * sample_rate:
        recording = False
    #print sample_counter
    time.sleep(0.1)
# Close all open streams and files
recorder.close()
pa.terminate()
wavefile.close()

Collected at 20160604193459917757.wav, 98 samples, with 21 silences
Collected at 20160604195322817529.wav, 68 samples, with 21 silences
Collected at 20160604200412699853.wav, 87 samples, with 21 silences
Collected at 20160604202625989934.wav, 115 samples, with 21 silences
Collected at 20160604204707085962.wav, 43 samples, with 21 silences
Collected at 20160604204832089342.wav, 46 samples, with 21 silences
Collected at 20160604205501247171.wav, 61 samples, with 21 silences
Collected at 20160604205509437954.wav, 52 samples, with 21 silences
Collected at 20160604205635060314.wav, 58 samples, with 21 silences
Collected at 20160604210041403211.wav, 114 samples, with 21 silences
Collected at 20160604210420268946.wav, 48 samples, with 21 silences
Collected at 20160604210526747701.wav, 51 samples, with 21 silences
Collected at 20160604210602320539.wav, 87 samples, with 21 silences
Collected at 20160604212837182933.wav, 51 samples, with 21 silences
Collected at 20160604212906755573.wav, 65 samp

In [9]:
#Create Data Frame for Log File
audio = pd.DataFrame({"RMS":rms_list,"TimeStamp":tstamp_list,"Recording":recording_list})
audio.head(20)

,RMS,Recording,TimeStamp
0,0.002701,False,20160604192645898405
1,0.001378,False,20160604192646898496
2,0.003121,False,20160604192647898522
3,0.001109,False,20160604192648898506
4,0.001273,False,20160604192649898639
5,0.001297,False,20160604192650898770
6,0.001239,False,20160604192651898779
7,0.001376,False,20160604192652898740
8,0.001240,False,20160604192653898801
9,0.001952,False,20160604192654898891


In [10]:
#Create Log File
audio.to_csv('RecordedAudio.csv')